In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import os
import glob
import cv2
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [2]:
import pathlib

dataset_dir = "/kaggle/input/plant-village-400/plant_village_400_data"
data_dir = pathlib.Path(dataset_dir).with_suffix('')

classes = os.listdir(dataset_dir)

In [3]:
batch_size = 16
img_height = 256
img_width = 256

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 14198 files belonging to 38 classes.
Using 11359 files for training.


In [5]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 14198 files belonging to 38 classes.
Using 2839 files for validation.


In [6]:
class_names = train_ds.class_names
print(class_names)

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Sp

In [7]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(16, 256, 256, 3)
(16,)


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
normalization_layer = layers.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

In [ ]:
num_classes = len(class_names)

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

@tf.keras.utils.register_keras_serializable()
class ClassToken(layers.Layer):
    def __init__(self):
        super(ClassToken, self).__init__()

    def build(self, input_shape):
        w_init = tf.keras.initializers.RandomNormal()
        self.w = self.add_weight(
            shape=(1, 1, input_shape[-1]),
            initializer=w_init,
            trainable=True,
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        cls_token = tf.broadcast_to(self.w, [batch_size, 1, inputs.shape[-1]])
        return cls_token

    def get_config(self):
        config = super(ClassToken, self).get_config()
        return config

    @classmethod
    def from_config(cls, config):
        return cls()

def transformer_encoder(x, num_heads, hidden_dim, mlp_dim, dropout_rate):
    skip = x
    x = layers.LayerNormalization()(x)
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=hidden_dim)(x, x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Add()([x, skip])

    skip = x
    x = layers.LayerNormalization()(x)
    x = layers.Dense(mlp_dim, activation="gelu")(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(hidden_dim)(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Add()([x, skip])

    return x

def create_mobilenet_vit_model(config):
    base_model = applications.MobileNetV3Small(
        input_shape=(config["image_size"], config["image_size"], config["num_channels"]),
        include_top=False,
        weights="imagenet"
    )
    base_model.trainable = False

    inputs = layers.Input(shape=(config["image_size"], config["image_size"], config["num_channels"]))
    x = base_model(inputs, training=False)

    h, w, c = x.shape[1], x.shape[2], x.shape[3]
    x = layers.Reshape((h * w, c))(x)

    x = layers.Dense(config["hidden_dim"])(x)

    class_token = ClassToken()(x)
    x = layers.Concatenate(axis=1)([class_token, x])

    positions = tf.range(start=0, limit=x.shape[1], delta=1)
    pos_embedding = layers.Embedding(input_dim=x.shape[1], output_dim=x.shape[-1])(positions)
    x = x + pos_embedding

    for _ in range(config["num_layers"]):
        x = transformer_encoder(x, config["num_heads"], config["hidden_dim"], config["mlp_dim"], config["dropout_rate"])

    x = layers.LayerNormalization()(x)
    x = x[:, 0, :]
    outputs = layers.Dense(config["num_classes"], activation="softmax")(x)

    model = models.Model(inputs, outputs)
    return model

def compile_model(model):
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def create_callbacks():
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
    return [early_stopping, lr_scheduler]

def train_model(model, train_ds, val_ds, config):
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=config["epochs"],
        callbacks=create_callbacks(),
        verbose=1
    )
    return history

if __name__ == "__main__":
    config = {
        "image_size": 256,
        "num_channels": 3,
        "num_classes": 38,
        "num_layers": 2,
        "hidden_dim": 256,
        "mlp_dim": 512,
        "num_heads": 4,
        "dropout_rate": 0.2,
        "epochs": 20
    }

    model = create_mobilenet_vit_model(config)
    model = compile_model(model)

/opt/conda/lib/python3.10/site-packages/keras/src/applications/mobilenet_v3.py:449: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [10]:
# Train the model
history = train_model(model, train_ds, val_ds, config)

Epoch 1/20


I0000 00:00:1727508433.615553     115 service.cc:145] XLA service 0x7afe78006da0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727508433.615617     115 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  9/710 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.0276 - loss: 4.2287     

I0000 00:00:1727508448.213284     115 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


710/710 ━━━━━━━━━━━━━━━━━━━━ 62s 53ms/step - accuracy: 0.5520 - loss: 1.7142 - val_accuracy: 0.9401 - val_loss: 0.1964 - learning_rate: 1.0000e-04
Epoch 2/20
710/710 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.9260 - loss: 0.2572 - val_accuracy: 0.9482 - val_loss: 0.1628 - learning_rate: 1.0000e-04
Epoch 3/20
710/710 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.9539 - loss: 0.1566 - val_accuracy: 0.9577 - val_loss: 0.1345 - learning_rate: 1.0000e-04
Epoch 4/20
710/710 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.9670 - loss: 0.1062 - val_accuracy: 0.9609 - val_loss: 0.1365 - learning_rate: 1.0000e-04
Epoch 5/20
710/710 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.9798 - loss: 0.0722 - val_accuracy: 0.9634 - val_loss: 0.1214 - learning_rate: 1.0000e-04
Epoch 6/20
710/710 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.9844 - loss: 0.0541 - val_accuracy: 0.9609 - val_loss: 0.1407 - learning_rate: 1.0000e-04
Epoch 7/20
710/710 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accurac

In [12]:
model.save('cnn_vit_98.keras')

In [13]:
from IPython.display import FileLink
FileLink('cnn_vit_98.keras')

/kaggle/working/cnn_vit_98.keras

In [16]:
all_images = []
all_labels = []

for images, labels in val_ds:
    all_images.append(images)
    all_labels.append(labels)

all_images = np.concatenate(all_images)
all_labels = np.concatenate(all_labels)

In [18]:
predictions = model.predict(all_images)

y_pred = np.argmax(predictions, axis=1)

y_true = all_labels

89/89 ━━━━━━━━━━━━━━━━━━━━ 11s 68ms/step


In [19]:
# Classification report
print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

Classification Report:

                                                    precision    recall  f1-score   support

                                Apple___Apple_scab       0.94      0.97      0.96        78
                                 Apple___Black_rot       0.99      1.00      0.99        79
                          Apple___Cedar_apple_rust       1.00      0.97      0.99        38
                                   Apple___healthy       1.00      1.00      1.00        79
                               Blueberry___healthy       1.00      1.00      1.00        86
          Cherry_(including_sour)___Powdery_mildew       1.00      0.99      0.99        77
                 Cherry_(including_sour)___healthy       1.00      0.98      0.99        87
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot       0.95      0.86      0.90        71
                       Corn_(maize)___Common_rust_       1.00      1.00      1.00        78
               Corn_(maize)___Northern_Leaf_Blight     

In [29]:
batch_size = 16  # Define a batch size
predictions = []

# Iterate over the images in batches
for i in range(0, len(all_images), batch_size):
    batch = all_images[i:i + batch_size]
    preds = model.predict(batch)  # Predict for the current batch
    predictions.append(preds)

# Concatenate all predictions into a single array
predictions = np.concatenate(predictions, axis=0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━

In [30]:
# Get the predicted class indices
y_pred = np.argmax(predictions, axis=1)

# Directly use all_labels as y_true since it's already a NumPy array
y_true = all_labels

# Calculate log loss
log_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, predictions)
average_log_loss = tf.reduce_mean(log_loss).numpy()

print(f"Log Loss: {average_log_loss:.4f}")

Log Loss: 0.0948
